<a href="https://colab.research.google.com/github/Neuro-Algorithm/Mouse-Pose-Estimation/blob/main/Mouse_Behaviour_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Install the latest DLC (includes SuperAnimal)

!pip install --pre deeplabcut

# 2. RESTART RUNTIME (Runtime → Restart runtime)

In [2]:
from pathlib import Path
import deeplabcut

Loading DLC 3.0.0rc13...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [3]:
from google.colab import files

print("Upload your mouse video file:")
uploaded = files.upload()

for filepath, content in uploaded.items():
    print(f' Uploaded: "{filepath}" ({len(content):,} bytes)')
    video_path = Path(filepath).resolve()

# If NOT using Colab, just specify your video path:
# video_path = "/path/to/working/directory/MousePoseEstimation-Neuro-2026-01-12/videos/m3v1mp4.mp4"

Upload your mouse video file:


Saving t8SB0nr_JnsQzHam.mp4 to t8SB0nr_JnsQzHam.mp4
 Uploaded: "t8SB0nr_JnsQzHam.mp4" (290,666 bytes)


In [4]:
# For general quadrupeds (also works for mice):
# superanimal_name = "superanimal_quadruped"

# For mouse from top view:
superanimal_name = "superanimal_quadruped"  # Best for mouse tracking!
model_name = "hrnet_w32"  # Good accuracy
detector_name = "fasterrcnn_resnet50_fpn_v2"  # Good detection
pcutoff = 0.15  # Confidence threshold (lower = more points, higher = more confident points)

print(f"\n Using model: {superanimal_name}")
print(f" Network: {model_name}")
print(f" Detector: {detector_name}")
print(f" Confidence cutoff: {pcutoff}")


 Using model: superanimal_quadruped
 Network: hrnet_w32
 Detector: fasterrcnn_resnet50_fpn_v2
 Confidence cutoff: 0.15


In [ ]:
print("\n Analyzing video with SuperAnimal...")

deeplabcut.video_inference_superanimal(
    [video_path],
    superanimal_name,
    model_name=model_name,
    detector_name=detector_name,
    videotype="mp4",  # e.g., ".mp4"
    video_adapt=True,  # Adapts to your video
    scale_list=[],  # Auto-scale
    pcutoff=pcutoff,
)

print("Analysis complete!")


 Analyzing video with SuperAnimal...
Running video inference on [PosixPath('/content/t8SB0nr_JnsQzHam.mp4')] with superanimal_quadruped_hrnet_w32
Using pytorch for model hrnet_w32
Loading.... superanimal_quadruped_hrnet_w32


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


superanimal_quadruped_hrnet_w32.pt:   0%|          | 0.00/118M [00:00<?, ?B/s]

Loading.... superanimal_quadruped_fasterrcnn_resnet50_fpn_v2


superanimal_quadruped_fasterrcnn_resnet5(…):   0%|          | 0.00/173M [00:00<?, ?B/s]

Using /content/t8SB0nr_JnsQzHam.mp4 for video adaptation training
Processing video /content/t8SB0nr_JnsQzHam.mp4
Starting to analyze /content/t8SB0nr_JnsQzHam.mp4
Video metadata: 
  Overall # of frames:    514
  Duration of video [s]:  8.56
  fps:                    60.04
  resolution:             w=602, h=360

Running detector with batch size 1


 76%|███████▌  | 390/514 [29:04<09:34,  4.63s/it]

In [ ]:
videotype = video_path.suffix
scale_list = []

deeplabcut.video_inference_superanimal(
    [video_path],
    superanimal_name,
    model_name=model_name,
    detector_name=detector_name,
    videotype=videotype,
    video_adapt=True,
    scale_list=scale_list,
    pcutoff=pcutoff,
)

In [ ]:
from base64 import b64encode
from IPython.display import HTML
import glob


In [ ]:
# Find the labeled video
directory = video_path.parent
basename = video_path.stem
pattern = f"{basename}*{superanimal_name}*{detector_name}*{model_name}*_labeled_after_adapt.mp4"
matches = list(directory.glob(pattern))

if matches:
    labeled_video_path = matches[0]
    print(f"\n Displaying labeled video: {labeled_video_path.name}")

    view_video = open(labeled_video_path, "rb").read()
    data_url = "data:video/mp4;base64," + b64encode(view_video).decode()
    HTML(f"""
    <video width=600 controls>
          <source src="{data_url}" type="video/mp4">
    </video>
    """)
else:
    print(" Labeled video not found. Check the video directory.")

In [ ]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt

# Load predictions
df = pd.read_hdf(h5_file)

# Load a frame from the video
cap = cv2.VideoCapture(video_path)
cap.set(cv2.CAP_PROP_POS_FRAMES, 100)  # Frame 100
ret, frame = cap.read()
cap.release()

# Get predictions for frame 100
frame_data = df.iloc[100]

# Plot keypoints with high confidence
plt.figure(figsize=(10, 6))
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# Extract animal0 data
animal_data = frame_data['superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2_']['animal0']

# Plot each body part
for bodypart in ['nose', 'upper_jaw', 'left_ear', 'right_ear']:
    if bodypart in animal_data:
        x = animal_data[bodypart]['x']
        y = animal_data[bodypart]['y']
        likelihood = animal_data[bodypart]['likelihood']

        if likelihood > 0.5:  # Only show confident predictions
            plt.plot(x, y, 'ro', markersize=2)
            plt.text(x, y, bodypart, fontsize=2, color='red')

plt.title('Frame 100 - Detected Keypoints')
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"Nose confidence: {animal_data['nose']['likelihood']:.3f}")
print(f"Left ear confidence: {animal_data['left_ear']['likelihood']:.3f}")
print(f"Right ear confidence: {animal_data['right_ear']['likelihood']:.3f}")